<h1>All works at the MET - Federico Zeri Catalog:</h1>
<p>We retrieve all the artworks currently preserved at the MET through the Zeri Catalog Endpoint</p>

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import sparql_dataframe
import pandas as pd

endpoint = "http://data.fondazionezeri.unibo.it/sparql"

query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>
SELECT DISTINCT ?artwork ?label_artwork ?inventory
WHERE {
  ?artwork crm:P50_has_current_keeper <https://w3id.org/zericatalog/the-metropolitan-museum-of-art-new-york-ny> .
  ?artwork  rdfs:label ?label_artwork.
  ?artwork crm:P55_has_current_location ?location .
  ?location  crm:P3_has_note ?inventory . 
  FILTER(LANG(?label_artwork) = "" || LANGMATCHES(LANG(?label_artwork), "en"))

}
"""

df = sparql_dataframe.get(endpoint, query, post=True)
df

,artwork,label_artwork,inventory
0,https://w3id.org/zericatalog/artwork/18789/item,"Artwork ""Luciani Sebastiano , Ritratto di Cris...",inv. 00.18.2
1,https://w3id.org/zericatalog/artwork/16633/item,"Artwork ""Del Conte Jacopino , Sacra Famiglia"" ...","inv. 1976.100.15, lascito Harry G. Sperling"
2,https://w3id.org/zericatalog/artwork/18917/item,"Artwork ""Anonimo sec. XVI , Ritratto di monaco...",inv. 1986.339.1
3,https://w3id.org/zericatalog/artwork/18008/item,"Artwork ""Rossi Francesco de' , Ritratto maschi...",inv. 45.128.11
4,https://w3id.org/zericatalog/artwork/18022/item,"Artwork ""Rossi Francesco de' , Ritratto maschi...",inv. 55.14
...,...,...,...
103,https://w3id.org/zericatalog/artwork/50670/item,"Artwork ""Bonvicino Alessandro , Ritratto di ge...",inv. 28.79
104,https://w3id.org/zericatalog/artwork/49125/item,"Artwork ""Tisi Benvenuto , San Nicola da Tolent...",inv. 17.190.24
105,https://w3id.org/zericatalog/artwork/49129/item,"Artwork ""Tisi Benvenuto , San Nicola da Tolent...",inv. 17.190.23
106,https://w3id.org/zericatalog/artwork/50819/item,"Artwork ""Bonvicino Alessandro , Deposizione di...",inv. 12.61


<p>We retrieve all the information about each artork's transfers</p>

In [2]:
#Works' History - Federico Zeri Catalog

URI_string = ""

for index, row in df.iterrows():
    if len(URI_string) == 0:
        URI_string = "?artwork = <" + row["artwork"] + ">"
    else: 
        URI_string = URI_string + "||" + "?artwork = <" + row["artwork"] + ">"

final_query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>
SELECT ?artwork  (count(distinct ?trasferimento_global) AS ?history) (group_concat(distinct ?label_receiver ; separator = ", ") AS ?receivers) (group_concat(distinct ?inventory ; separator = ", ") AS ?id) (group_concat(distinct ?surrender ; separator = ", ") AS ?surrenders)          
WHERE {{
  ?trasferimento_global crm:P30_transferred_custody_of ?artwork .
  OPTIONAL {{ ?trasferimento_global crm:P28_custody_surrendered_by ?surrender . }}
  ?trasferimento_global crm:P28_custody_surrendered_by ?transf .
  ?receiver crm:P29i_received_custody_through ?trasferimento_global .
  ?artwork crm:P50_has_current_keeper <https://w3id.org/zericatalog/the-metropolitan-museum-of-art-new-york-ny> .
  ?artwork crm:P55_has_current_location ?location .
  ?location  crm:P3_has_note ?inventory .
  FILTER ({0})
  
  ?artwork  rdfs:label ?label_artwork.
  ?surrender rdfs:label ?label_surrender.
  ?receiver rdfs:label ?label_receiver.
  
  FILTER(?surrender != <https://w3id.org/zericatalog/>)
  
  FILTER(LANG(?label_artwork) = "" || LANGMATCHES(LANG(?label_artwork), "en"))
  FILTER(LANG(?label_surrender) = "" || LANGMATCHES(LANG(?label_surrender), "en"))
  FILTER(LANG(?label_receiver) = "" || LANGMATCHES(LANG(?label_receiver), "en"))
}} GROUP BY ?artwork
"""

final_df = sparql_dataframe.get(endpoint, final_query.format(URI_string), post=True)
final_df

,artwork,history,receivers,id,surrenders
0,https://w3id.org/zericatalog/artwork/39117/item,4,"The Metropolitan Museum of Art, Asta Christie'...",inv. 1970.134.1,https://w3id.org/zericatalog/asta-sothebys-lon...
1,https://w3id.org/zericatalog/artwork/39828/item,5,"The Metropolitan Museum of Art, E. Sestieri, C...",inv. 48.17.12,"https://w3id.org/zericatalog/e-sestieri-roma, ..."
2,https://w3id.org/zericatalog/artwork/39818/item,5,"The Metropolitan Museum of Art, E. Sestieri, C...",inv. 48.17.5,"https://w3id.org/zericatalog/e-sestieri-roma, ..."
3,https://w3id.org/zericatalog/artwork/45055/item,1,The Metropolitan Museum of Art,"inv. 14.40.650, dono B. Altman",https://w3id.org/zericatalog/collezione-b-altm...
4,https://w3id.org/zericatalog/artwork/39821/item,5,"The Metropolitan Museum of Art, E. Sestieri, C...",inv. 48.17.7,"https://w3id.org/zericatalog/e-sestieri-roma, ..."
5,https://w3id.org/zericatalog/artwork/39831/item,5,"The Metropolitan Museum of Art, E. Sestieri, C...",inv. 48.17.14,"https://w3id.org/zericatalog/e-sestieri-roma, ..."
6,https://w3id.org/zericatalog/artwork/16633/item,4,"The Metropolitan Museum of Art, Collezione H.G...","inv. 1976.100.15, lascito Harry G. Sperling",https://w3id.org/zericatalog/collezione-hg-spe...
7,https://w3id.org/zericatalog/artwork/41423/item,2,"The Metropolitan Museum of Art, Asta Christie's",inv. 1973.311.1,https://w3id.org/zericatalog/asta-christies-lo...
8,https://w3id.org/zericatalog/artwork/60085/item,1,The Metropolitan Museum of Art,inv. 1971.115.3b,https://w3id.org/zericatalog/duveen-brothers-i...
9,https://w3id.org/zericatalog/artwork/39841/item,5,"The Metropolitan Museum of Art, E. Sestieri, C...",inv. 48.17.20,"https://w3id.org/zericatalog/e-sestieri-roma, ..."


In [3]:
a_list = []

for idx, row in final_df.iterrows():
    a_list.append(str(row["id"]).replace("inv. ", "").split(",")[0])
a_list
final_df["Object Number"] = a_list
final_df = final_df.drop(['id'], axis=1)


ser = final_df["Object Number"]
ser = ser.tolist()
ser
id_dict = {}
a_list = []

for el in ser:
    w = str()
    for inn in el.split(".")[:-1]:
        if len(w) > 0:
            w = w + "." + inn
        else:
            w = w + inn
    if w not in id_dict:
        id_dict[w] = [el]
    else:
        a = id_dict[w]
        a.append(el)
        id_dict[w] = a

In [4]:
artwork_uris = []
result_df = final_df

number_rows = final_df.shape[0]

for el in id_dict["48.17"]:
    for idx, row in final_df.iterrows():
        if el == row["Object Number"]:
            history = row["history"]
            receivers = row["receivers"]
            surrenders = row["surrenders"]
            artwork_uris.append(row["artwork"])
            result_df = result_df.drop([idx])

result_df.loc[number_rows, :] = [(", ".join(artwork_uris)), history, receivers, surrenders, "48.17.1–22"]
number_rows = number_rows + 1

artwork_uris = []
for el in id_dict["1971.115"]:
    for idx, row in final_df.iterrows():
        if el == row["Object Number"]:
            history = row["history"]
            receivers = row["receivers"]
            surrenders = row["surrenders"]
            artwork_uris.append(row["artwork"])
            result_df = result_df.drop([idx])
            
result_df.loc[number_rows, :] = [(", ".join(artwork_uris)), history, receivers, surrenders, "1971.115.3ab"]
result_df

,artwork,history,receivers,surrenders,Object Number
0,https://w3id.org/zericatalog/artwork/39117/item,4.0,"The Metropolitan Museum of Art, Asta Christie'...",https://w3id.org/zericatalog/asta-sothebys-lon...,1970.134.1
3,https://w3id.org/zericatalog/artwork/45055/item,1.0,The Metropolitan Museum of Art,https://w3id.org/zericatalog/collezione-b-altm...,14.40.650
6,https://w3id.org/zericatalog/artwork/16633/item,4.0,"The Metropolitan Museum of Art, Collezione H.G...",https://w3id.org/zericatalog/collezione-hg-spe...,1976.100.15
7,https://w3id.org/zericatalog/artwork/41423/item,2.0,"The Metropolitan Museum of Art, Asta Christie's",https://w3id.org/zericatalog/asta-christies-lo...,1973.311.1
10,https://w3id.org/zericatalog/artwork/32096/item,4.0,"The Metropolitan Museum of Art, P. Corsini, Co...",https://w3id.org/zericatalog/p-corsini-new-yor...,1984.74
11,https://w3id.org/zericatalog/artwork/45062/item,4.0,"The Metropolitan Museum of Art, Collezione J. ...",https://w3id.org/zericatalog/collezione-j-pier...,16.30ab
12,https://w3id.org/zericatalog/artwork/46494/item,1.0,The Metropolitan Museum of Art,https://w3id.org/zericatalog/collezione-g-e-f-...,41.100.12
13,https://w3id.org/zericatalog/artwork/18022/item,3.0,"The Metropolitan Museum of Art, Collezione J.H...",https://w3id.org/zericatalog/collezione-jh-har...,55.14
18,https://w3id.org/zericatalog/artwork/36357/item,3.0,"The Metropolitan Museum of Art, Collezione T.M...",https://w3id.org/zericatalog/collezione-tm-dav...,30.95.270
21,https://w3id.org/zericatalog/artwork/45063/item,1.0,The Metropolitan Museum of Art,https://w3id.org/zericatalog/convento-di-s-ant...,32


In [5]:
from pandas import read_sql
from sqlite3 import connect

csvData = pd.read_csv("https://media.githubusercontent.com/media/metmuseum/openaccess/master/MetObjects.csv", encoding='utf-8')
with connect("metDB.db") as con:
    csvData.to_sql("MetCatalog", con, if_exists="replace", index=False)
    con.commit()
with connect("metDB.db") as con:
    query = """
        SELECT *
        FROM `MetCatalog`
        WHERE ("Artist Nationality" LIKE '%talian%' OR "Artist Display Bio" LIKE '%talian%') AND "Department" = "European Paintings";
    """
    italianDataset = pd.read_sql(query, con)



C:\Users\sebas\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5,7,10,11,12,13,14,34,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\sebas\anaconda3\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [6]:
#italianDataset.to_csv("ita.csv")

In [7]:
#merging MET and Federico Zeri

from pandas import merge


result = pd.merge(result_df, italianDataset)
result.to_csv("result.csv")
result

,artwork,history,receivers,surrenders,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,...,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
0,https://w3id.org/zericatalog/artwork/39117/item,4.0,"The Metropolitan Museum of Art, Asta Christie'...",https://w3id.org/zericatalog/asta-sothebys-lon...,1970.134.1,0,0,1,436567,None,...,None,Paintings,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19911541,None,"Metropolitan Museum of Art, New York, NY",Infants|Women|Saint John the Baptist|Bedrooms|...,http://vocab.getty.edu/page/aat/300189561|http...,https://www.wikidata.org/wiki/Q998|https://www...
1,https://w3id.org/zericatalog/artwork/45055/item,1.0,The Metropolitan Museum of Art,https://w3id.org/zericatalog/collezione-b-altm...,14.40.650,0,1,1,437823,608.0,...,None,Paintings,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19904807,None,"Metropolitan Museum of Art, New York, NY",Men|Portraits,http://vocab.getty.edu/page/aat/300025928|http...,https://www.wikidata.org/wiki/Q8441|https://ww...
2,https://w3id.org/zericatalog/artwork/16633/item,4.0,"The Metropolitan Museum of Art, Collezione H.G...",https://w3id.org/zericatalog/collezione-hg-spe...,1976.100.15,0,0,1,436770,None,...,None,Paintings,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19912153,None,"Metropolitan Museum of Art, New York, NY",Holy Family|Saint Joseph|Christ|Virgin Mary,http://vocab.getty.edu/page/ia/901001175|http:...,https://www.wikidata.org/wiki/Q618057|https://...
3,https://w3id.org/zericatalog/artwork/41423/item,2.0,"The Metropolitan Museum of Art, Asta Christie's",https://w3id.org/zericatalog/asta-christies-lo...,1973.311.1,0,1,1,435722,626.0,...,None,Paintings,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19905185,None,"Metropolitan Museum of Art, New York, NY",Armor|Boys|Men|Portraits,http://vocab.getty.edu/page/aat/300226591|http...,https://www.wikidata.org/wiki/Q20793164|https:...
4,https://w3id.org/zericatalog/artwork/32096/item,4.0,"The Metropolitan Museum of Art, P. Corsini, Co...",https://w3id.org/zericatalog/p-corsini-new-yor...,1984.74,0,1,1,437342,None,...,None,Paintings,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19911628,None,"Metropolitan Museum of Art, New York, NY",Sadness|Suffering|Mary Magdalene|Saint John th...,http://vocab.getty.edu/page/aat/300055166|http...,https://www.wikidata.org/wiki/Q169251|https://...
5,https://w3id.org/zericatalog/artwork/45062/item,4.0,"The Metropolitan Museum of Art, Collezione J. ...",https://w3id.org/zericatalog/collezione-j-pier...,16.30ab,1,1,1,437372,962.0,...,None,Paintings,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q27033728,None,"Metropolitan Museum of Art, New York, NY",Madonna and Child|Saint Catherine|Saint John t...,http://vocab.getty.edu/page/ia/901000052| |htt...,https://www.wikidata.org/wiki/Q9309699| |https...
6,https://w3id.org/zericatalog/artwork/46494/item,1.0,The Metropolitan Museum of Art,https://w3id.org/zericatalog/collezione-g-e-f-...,41.100.12,0,1,1,437822,608.0,...,None,Paintings,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19905075,None,"Metropolitan Museum of Art, New York, NY",Men|Portraits,http://vocab.getty.edu/page/aat/300025928|http...,https://www.wikidata.org/wiki/Q8441|https://ww...
7,https://w3id.org/zericatalog/artwork/18022/item,3.0,"The Metropolitan Museum of Art, Collezione J.H...",https://w3id.org/zericatalog/collezione-jh-har...,55.14,0,1,1,437598,None,...,None,Paintings,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19912783,None,"Metropolitan Museum of Art, New York, NY",Men|Portraits,http://vocab.getty.edu/page/aat/300025928|http.

In [8]:
import re

new_string = ""
for idx, row in result.iterrows():
    if idx == 0:
        if "," not in row["artwork"]:
            new_string = new_string + "strstarts(str(?custody), '" + row["artwork"].replace("/item", "/custody") + "')"
        else:
            artw = row["artwork"].split(",")[0]
            new_string = new_string + "strstarts(str(?custody), '" + artw.replace("/item", "/custody") + "')"
    else:
        if "," not in row["artwork"]:
            new_string = new_string + "|| strstarts(str(?custody), '" + row["artwork"].replace("/item", "/custody") + "')"
        else: 
            artw = row["artwork"].split(",")[0]
            new_string = new_string + "|| strstarts(str(?custody), '" + artw.replace("/item", "/custody") + "')"
access = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT DISTINCT ?custody ?time_span ?start_date ?end_date ?place_label ?label_receiver ?location
WHERE {{
  ?receiver crm:P29i_received_custody_through ?custody .
  ?receiver crm:P74_has_current_or_former_residence ?place .
  ?custody crm:P4_has_time_span ?time_span .
  ?time_span ti:hasIntervalEndDate ?end_date .
  ?time_span ti:hasIntervalStartDate ?start_date .
  ?place owl:sameAs ?location .
  ?place rdfs:label ?place_label .
  ?receiver rdfs:label ?label_receiver .
  FILTER(LANG(?place_label) = "" || LANGMATCHES(LANG(?place_label), "en"))
  FILTER(LANG(?label_receiver) = "" || LANGMATCHES(LANG(?label_receiver), "en"))
  FILTER ({0})
}}
"""
third_df = sparql_dataframe.get(endpoint, access.format(new_string), post=True)   
third_df.to_csv("final.csv")

save = italianDataset

<h1>Credit Line - MET Dataset</h1>

In [9]:
empty = pd.Series([""])
resp = pd.Series([""])

def add_last_transfer(num, obj):
    italianDataset.at[num, "lastTransfer"] = obj

def add_agent(num, agent):
    if agent.startswith("and "):
        agent = agent.replace("and ", "")
    if agent.endswith(" "):
        agent = agent[:-1]
    if type(italianDataset.at[num, "agentsTransfer"]) == type("") and len(italianDataset.at[num, "agentsTransfer"]) > 0:
        if agent not in italianDataset.at[num, "agentsTransfer"].split(", ") and (agent + " Fund") not in italianDataset.at[num, "agentsTransfer"].split(", "):
            italianDataset.at[num, "agentsTransfer"] = italianDataset.at[num, "agentsTransfer"] + ", " + agent
        else:
            return None
    else:
        italianDataset.at[num, "agentsTransfer"] = agent

def and_case(a_string, credit):
    if " and " in credit and "Mr" not in credit and "Mrs." not in credit and "Foundation" not in credit and "Fund" not in credit and "Collection" not in credit and "Gallery" not in credit and "Charitable" not in credit:
        x = modified_credit.replace(a_string, "").split(" and ")
        if " " in x[0] and " " in x[0]:
            f = x[0]
            s = x[1]
            add_agent(idx, f)
            add_agent(idx, s)
        else: 
            add_agent(idx, modified_credit.replace(a_string, ""))
    else:
        add_agent(idx, modified_credit.replace(a_string, ""))

    


a = 0
while a < 2:
    for idx, row in italianDataset.iterrows():
        if row["Credit Line"].count(";") > 1:
            italianDataset.at[idx, "Credit Line"]=row["Credit Line"].replace(";", ",")
        if row["Credit Line"].startswith("The Metropolitan Museum of Art, New York."):
            italianDataset.at[idx, "Credit Line"] = row["Credit Line"].replace("The Metropolitan Museum of Art, New York. ", "")
        if ":" in row["Credit Line"]:
            x = row["Credit Line"]
            x = x.replace(":",  ",")
            italianDataset.at[idx, "Credit Line"] = x
        if ", by exchange," in row["Credit Line"]:
            italianDataset.at[idx, "Credit Line"] = row["Credit Line"].replace(", by exchange",  "")
    a = a + 1
italianDataset["lastTransfer"] = empty
italianDataset["agentsTransfer"] = resp
for idx, row in italianDataset.iterrows():        
    if ";" not in row["Credit Line"]:
        credit_line = row["Credit Line"].split(", ")
        modified_credit = credit_line[0]
        if modified_credit == "Anonymous Gift" and len(credit_line) == 2 or row["Credit Line"].startswith("Anonymous Gift, in memory"):
            add_last_transfer(idx, "Gift")
        if modified_credit.startswith("Bequest of "):
            add_last_transfer(idx, "Bequest")
            if " in honor of" in modified_credit:
                modified_credit = modified_credit.split(" in honor of")[0]
            and_case("Bequest of ", modified_credit)
        elif modified_credit.startswith("The Bequest of "):
            add_last_transfer(idx, "Bequest")
            and_case("The Bequest of ", modified_credit)
        elif modified_credit.startswith("The Collection of ") or modified_credit.startswith("From the Collection of "):
            if modified_credit.startswith("From the Collection of "):
                modified_credit = modified_credit.replace ("From ", "")
            add_agent(idx, modified_credit)
            add_last_transfer(idx, "Collection")
        elif modified_credit.startswith("Gift of "):
            add_last_transfer(idx, "Gift")
            and_case("Gift of ", modified_credit)
            for n in range(len(credit_line)-2):
                n = n + 1
                if "Purchase" not in credit_line[n] and "Collection" not in credit_line[n] and "Bequest" not in credit_line[n] and "Fund" not in credit_line[n] and "Gift" not in credit_line[n] and "memory" not in credit_line[n] and "in honor" not in credit_line[n] and "in celebration" not in credit_line[n]:
                    if "in memory of h" not in credit_line[n-1] and "in honor" not in credit_line[n-1]:
                        add_agent(idx, credit_line[n].replace("and ", "", 1))
        elif modified_credit.startswith("Purchase"):
            add_last_transfer(idx, "Purchase")
        elif modified_credit.endswith("Collection") or modified_credit.endswith("Gallery"):
            add_agent(idx, modified_credit)
            add_last_transfer(idx, "Collection")
        elif modified_credit.endswith("Fund") or modified_credit.endswith("Trust"):
            add_agent(idx, modified_credit)
            add_last_transfer(idx, "Purchase")
        elif modified_credit.endswith("Funds") and " and " in modified_credit:
            add_last_transfer(idx, "Purchase")
            for el in modified_credit.split(" and "):
                add_agent(idx, (el + " Fund"). replace(" Funds", ""))
        elif "Funds" in row["Credit Line"] and "Bequest" not in row["Credit Line"] and "Purchase" not in row["Credit Line"] and "Gift" not in row["Credit Line"] and "funds from various donors" not in row["Credit Line"]:
            add_last_transfer(idx, "Purchase")
            split = row["Credit Line"].split(",")
            if len(split) > 2:
                for n in range(len(split)-1):
                    a = split[n].replace(" Funds", "") + " Fund"
                    if a.startswith(" and"):
                        a =a.replace(" and ", "")
                    elif a.startswith(" "):
                        a =a.replace (" ", "", 1)
                    add_agent(idx, a)
            else:
                if split[0] == "Funds from various donors":
                    italianDataset.at[idx, "agentsTransfer"] = italianDataset.at[idx, "agentsTransfer"]
                else:
                    new_split = split[0].split(" and ")
                    for n in range(len(new_split)):
                        a = new_split[n].replace(" Funds", "") + " Fund"
                        add_agent(idx, a)       
        if len(credit_line) > 2:
            for n in range(len(credit_line)-2):
                num = n + 1
                if credit_line[num] and "in memory " not in credit_line[num]:
                    new_modified_credit = credit_line[num]
                    if new_modified_credit.startswith("Bequest and Gift"):
                        add_agent(idx, new_modified_credit.replace("Bequest and Gift of ", ""))
                    if new_modified_credit.startswith("Bequest of"):
                        if " and " in new_modified_credit and "Mr" not in new_modified_credit and "Mrs." not in new_modified_credit and "Foundation" not in new_modified_credit and "Fund" not in new_modified_credit and "Collection" not in new_modified_credit and "Gallery" not in new_modified_credit and "Charitable" not in new_modified_credit:
                            if " and Gift " in new_modified_credit:
                                x = new_modified_credit.replace("Bequest of ", "").split(" and Gift of ")
                                add_agent(idx, x[0])
                                add_agent(idx, x[1])
                            else:
                                x = new_modified_credit.replace("Bequest of ", "").split(" and ")
                                if " " in x[0] and " " in x[0]:
                                    f = x[0]
                                    s = x[1]
                                    add_agent(idx, f)
                                    add_agent(idx, s)
                                else: 
                                    add_agent(idx, new_modified_credit.replace(" Bequest of ", ""))
                        else:
                            add_agent(idx, new_modified_credit.replace("Bequest of ", ""))
                    if new_modified_credit.startswith("Bequests of"):
                        if " and " in new_modified_credit and "Mr" not in new_modified_credit and "Mrs." not in new_modified_credit and "Foundation" not in new_modified_credit and "Fund" not in new_modified_credit and "Collection" not in new_modified_credit and "Gallery" not in new_modified_credit and "Charitable" not in new_modified_credit:
                            x = new_modified_credit.replace("Bequests of ", "").split(" and ")
                            if " " in x[0] and " " in x[0]:
                                f = x[0]
                                s = x[1]
                                add_agent(idx, f)
                                add_agent(idx, s) 
                            else: 
                                add_agent(idx, new_modified_credit.replace("Bequest of ", ""))
                        else:
                            add_agent(idx, new_modified_credit.replace("Bequests of ", ""))
                        new_n = num + 1
                        while new_n != 0:
                            if not credit_line[new_n].startswith("1") and "Bequest" not in credit_line[new_n] and "Gift" not in credit_line[new_n] and "Purchase" not in credit_line[new_n] and "Fund" not in credit_line[new_n] and "Collection" not in credit_line[new_n]:
                                if "in memory " not in credit_line[new_n] and "in memory of h" not in credit_line[new_n-1]:
                                    add_agent(idx, credit_line[new_n])
                                new_n = new_n + 1
                            else:
                                new_n = 0
                    if new_modified_credit.startswith("and Gifts of"):
                        if " and " in new_modified_credit:
                            x = new_modified_credit.split(" and ")
                            for el in x:
                                add_agent(idx, el.replace("and Gifts of ", ""))
                    if new_modified_credit.startswith("and Bequests of"):
                        if " and " in new_modified_credit and "Mr" not in new_modified_credit and "Mrs." not in new_modified_credit and "Foundation" not in new_modified_credit and "Fund" not in new_modified_credit and "Collection" not in new_modified_credit and "Gallery" not in new_modified_credit and "Charitable" not in new_modified_credit:
                            x = new_modified_credit.replace("and Bequests of ", "").split(" and ")
                            if " " in x[0] and " " in x[0]:
                                f = x[0]
                                s = x[1]
                                add_agent(idx, f)
                                add_agent(idx, s) 
                            else: 
                                add_agent(idx, new_modified_credit.replace(" Bequest of ", ""))
                        else:
                            add_agent(idx, new_modified_credit.replace("and Bequests of ", ""))
                    if new_modified_credit.startswith("and Bequest of"):
                        add_agent(idx, new_modified_credit.replace("and Bequest of ", ""))
                    if new_modified_credit.startswith("Gift of"):
                        if " and " in new_modified_credit and "Mr" not in modified_credit and "Mrs." not in new_modified_credit and "Foundation" not in new_modified_credit and "Fund" not in new_modified_credit and "Collection" not in new_modified_credit and "Gallery" not in new_modified_credit and "Charitable" not in new_modified_credit and "and Bequest" not in new_modified_credit:
                            x = new_modified_credit.replace("Gift of ", "").split(" and ")
                            if " " in x[0] and " " in x[0]:
                                f = x[0]
                                s = x[1]
                                add_agent(idx, f)
                                add_agent(idx, s) 
                            else: 
                                add_agent(idx, new_modified_credit.replace(" Gift of ", ""))
                        elif " and Bequest of " in new_modified_credit:
                            if new_modified_credit.startswith("Gift and"):
                                new_modified_credit = new_modified_credit.replace("Gift", "")
                            x = new_modified_credit.split(" and Bequest of ")
                            for el in x:
                                add_agent(idx, el.replace("Gift of ", ""))
                        else:
                            add_agent(idx, new_modified_credit.replace("Gift of ", ""))
                    if new_modified_credit.startswith("and Gift "):
                        add_agent(idx, new_modified_credit.replace("and Gift of ", ""))
                    if new_modified_credit.endswith("Gift"):
                        if " and " in new_modified_credit and "Mr" not in modified_credit and "Mrs." not in new_modified_credit and "Foundation" not in new_modified_credit and "Fund" not in new_modified_credit and "Collection" not in new_modified_credit and "Gallery" not in new_modified_credit and "Charitable" not in new_modified_credit and "and Bequest" not in new_modified_credit:
                            x = new_modified_credit.replace("Gift", "").split(" and ")
                            if " " in x[0] and " " in x[0]:
                                f = x[0]
                                s = x[1]
                                add_agent(idx, f)
                                add_agent(idx, s) 
                            else: 
                                if new_modified_credit.replace("Gift", "").startswith(" and"):
                                    add_agent(idx, new_modified_credit.replace("Gift", "").replace("and ", ""))
                                else:
                                    add_agent(idx, new_modified_credit.replace("Gift", ""))
                        elif "Fund" in new_modified_credit and " and " in new_modified_credit:
                            x = new_modified_credit.split(" and ")
                            for el in x:
                                add_agent(idx, el.replace(" Gift", "")) 
                        else:
                            a = new_modified_credit.replace(" Gift", "")
                            if a.startswith("and "):                       
                                a=a.replace("and ", "", 1)
                            add_agent(idx, a)
                    if new_modified_credit.endswith("Bequest"):
                        if " and " in new_modified_credit and "Mr" not in modified_credit and "Mrs." not in new_modified_credit and "Foundation" not in new_modified_credit and "Fund" not in new_modified_credit and "Collection" not in new_modified_credit and "Gallery" not in new_modified_credit and "Charitable" not in new_modified_credit and "and Bequest" not in new_modified_credit:
                            x = new_modified_credit.replace("Bequest", "").split(" and ")
                            if " " in x[0] and " " in x[0]:
                                f = x[0]
                                s = x[1]
                                add_agent(idx, f)
                                add_agent(idx, s) 
                            else: 
                                add_agent(idx, new_modified_credit.replace("Gift", ""))
                        else:
                            add_agent(idx, new_modified_credit.replace("Bequest", ""))
                    if new_modified_credit.endswith("Fund"):
                        if "Gift" in new_modified_credit and " and " in new_modified_credit and "and Bequest of" not in new_modified_credit:
                            x = new_modified_credit.split(" and ")
                            for el in x:
                                add_agent(idx, el.replace(" Gift", "")) 
                        elif "Gift and Bequest of" in new_modified_credit:
                            x = new_modified_credit.replace("Gift and Bequest of ", "").split(" and ")
                            for el in x:
                                add_agent(idx, el)
                        else:
                            add_agent(idx, new_modified_credit)
                    if new_modified_credit.endswith("Funds"):
                        y = num
                        if "Purchase" not in credit_line[y-1] and "Gift" not in credit_line[y-1] and "Bequest" not in credit_line[y-1] and "Collection" not in credit_line[y-1]:
                            add_agent(idx, new_modified_credit.replace(" Funds", " Fund").replace("and ", "", 1))
                            
                            while y > 1:
                                y = y - 1
                                if "Purchase" not in credit_line[y] and "Gift" not in credit_line[y] and "Bequest" not in credit_line[y] and "Collection" not in credit_line[y] and "in memory" not in credit_line[y]:
                                    if credit_line[y].startswith("and "):
                                        add_agent(idx, (credit_line[y] + " Fund").replace("and ", "", 1))
                                    else:
                                        add_agent(idx, (credit_line[y] + " Fund"))
                                else:
                                    y = 0
                        else:
                            if " and " in new_modified_credit:
                                for el in new_modified_credit.split(" and "):
                                    add_agent(idx, (el.replace(" Funds", "") + " Fund"))
                    if new_modified_credit.endswith(" Gifts"):
                        
                        if new_modified_credit.count(" and ") > 1 and not new_modified_credit.startswith(" and "):
                            x = new_modified_credit.split(" and ")
                            for n in range(len(x)-1):
                                num = n + 1
                                if " " not in x[num-1] and " " in x[num]:
                                    f = x[num-1] + " and " + x[num]
                                    add_agent(idx, f.replace(" Gifts", ""))
                                else:
                                    add_agent(idx, x[num].replace(" Gifts", ""))
                        elif "Foundation" in new_modified_credit and " and " in new_modified_credit:
                            x = new_modified_credit.split(" and ")
                            for el in x:
                                add_agent(icredit.replace(" Gifts", ""))
                        else:
                            y = num
                            if new_modified_credit.startswith("and "):
                                new_modified_credit = new_modified_credit.replace("and ", "", 1)
                            add_agent(idx, new_modified_credit.replace(" Gifts", ""))
                            while y > 1:
                                y = y - 1
                                if "Purchase" not in credit_line[y] and "Gift" not in credit_line[y] and "Bequest" not in credit_line[y] and "Collection" not in credit_line[y]:
                                    add_agent(idx, credit_line[y])
                                else: 
                                    y = 0
                                
    elif row["Credit Line"].count(";") == 1:
        credit_line_update = row["Credit Line"].split(";")
        credit_line_segment_one = credit_line_update[0].split(",")
        if credit_line_segment_one[0].startswith("Gift of"):
            add_last_transfer(idx, "Gift")
            if " and " in credit_line_segment_one[0] and "Mr" not in credit_line_segment_one[0] and "Mrs." not in credit_line_segment_one[0] and "Foundation" not in credit_line_segment_one[0] and "Fund" not in credit_line_segment_one[0] and "Collection" not in credit_line_segment_one[0] and "Gallery" not in credit_line_segment_one[0] and "Charitable" not in credit_line_segment_one[0]:
                x = credit_line_segment_one[0].replace("Gift of ", "").split(" and ")
                if " " in x[0] and " " in x[0]:
                    f = x[0]
                    s = x[1]
                    add_agent(idx, f)
                    add_agent(idx, s) 
                else: 
                    add_agent(idx, credit_line_segment_one[0].replace("Gift of ", ""))
            else:
                add_agent(idx, credit_line_segment_one[0].replace("Gift of ", ""))
        if credit_line_segment_one[0].startswith("Purchase"):
            add_last_transfer(idx, "Purchase")
        if credit_line_segment_one[-1].endswith("Gifts"):

            for n in range(len(credit_line_segment_one) - 1):
                num = n + 1
                if "Collection" in credit_line_segment_one[num] or "Fund" in credit_line_segment_one[num] or "Foundation" in credit_line_segment_one[num]:
                    add_agent(idx, credit_line_segment_one[num].replace(" ", "", 1))
                else:
                    if credit_line_segment_one[num].startswith(" and"):
                        credit_line_segment_one[num] = credit_line_segment_one[num].replace(" and", "")
                    add_agent(idx, credit_line_segment_one[num].replace(" ", "", 1).replace(" Gifts", ""))
        credit_line_segment_two = credit_line_update[1].split(",")
        if credit_line_segment_two[0].startswith(" and Gifts"): 
            for n in range(len(credit_line_segment_two) - 1):
                num = n
                if "Bequest" not in credit_line_segment_two[num] and "Collection" not in credit_line_segment_two[num] and "Fund" not in credit_line_segment_two[num] and "by exchange" not in credit_line_segment_two[num]:
                    if "Mr. and Mrs." in credit_line_segment_two[num]:
                        add_agent(idx, credit_line_segment_two[num].replace(" and Gifts of","").replace(" ", "", 1))
                    else:
                        add_agent(idx, credit_line_segment_two[num].replace(" and Gifts of","").replace(" and ", " ", 1).replace(" ", "", 1))
                elif "Collection" in credit_line_segment_two[num] or "Fund" in credit_line_segment_two[num]:
                    add_agent(idx, credit_line_segment_two[num].replace(" and","").replace(" ", "", 1))
                elif "Bequest" in credit_line_segment_two[num]:
                    if credit_line_segment_two[num].startswith(" Bequests of"):
                        inner_list = credit_line_segment_two[num].split(" and")
                        for el in inner_list:
                            add_agent(idx, el.replace(" Bequests of","").replace(" ", "", 1))
        for el in credit_line_segment_two:
            if "Collection" in el:
                add_agent(idx, el.replace(" ", "", 1))

italianDataset
italianDataset.to_csv("transfers.csv")

In [10]:
# Further details might be extracted from the MET website:


#from urllib.request import urlopen
#from bs4 import BeautifulSoup


#number_rows = italianDataset.shape[0]
#for n in range(number_rows):
#    url = italianDataset.at[n, "Link Resource"]

#    print(url)
#    html = urlopen(url).read()
#    soup = BeautifulSoup(html, features="html.parser")
#    prov = soup.find(id="provenance").div.text
#    print(prov)